# Dengue no Recife: Exploração Inicial dos Dados

Este notebook faz parte do projeto **Dengue, Território e Condições Urbanas no Recife**, desenvolvido por mim, Gustavo Felipe, para analisar os dados recentes disponibilizados sobre dengue, zika e chikungunya no Recife e também alguns dados demográficos 
## Objetivo deste notebook

O objetivo desta etapa é realizar uma **exploração inicial (EDA)** dos dados disponíveis, com foco em:

- Compreender a estrutura dos dados de saúde (Dengue, Zika e Chikungunya)
- Identificar variáveis territoriais e temporais
- Avaliar qualidade, consistência e limitações das bases
- Explorar os dados demográficos do Censo 2022 (IBGE)

⚠️ Este notebook **não realiza ainda limpeza profunda nem integração final** entre as bases.
Essas etapas serão desenvolvidas em notebooks posteriores.


In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Configurações visuais
plt.style.use("default")
sns.set_context("notebook")

# Exibir todas as colunas
pd.set_option("display.max_columns", None)


## Organização dos dados

Os dados utilizados neste projeto estão organizados da seguinte forma:

- `data/raw/dengue/` → Casos de Dengue (2020–2024)
- `data/raw/zika/` → Casos de Zika (2020–2024)
- `data/raw/chikungunya/` → Casos de Chikungunya (2020–2024)
- `data/raw/tabelas_auxiliares/` → Bairros, distritos, agravos, UF
- `data/raw/metadados/` → Metadados em formato JSON
- `data/raw/demografia/` → Dados do Censo 2022 (IBGE)


In [4]:
# Função para ler arquivos CSV com configurações padrão para dados brasileiros
def read_csv_br(caminho):
    return pd.read_csv(
        caminho,
        sep=";",
        encoding="latin1",
        low_memory=False,
        on_bad_lines="skip"
    )

In [5]:
dengue_2020 = read_csv_br("../data/raw/dengue/dengon2020_recife.csv")
dengue_2020.head()


,nu_notificacao,tp_notificacao,co_cid,dt_notificacao,ds_semana_notificacao,notificacao_ano,co_uf_notificacao,co_municipio_notificacao,id_regional,co_unidade_notificacao,dt_diagnostico_sintoma,ds_semana_sintoma,dt_nascimento,nu_idade,tp_sexo,tp_gestante,tp_raca_cor,tp_escolaridade,co_uf_residencia,co_municipio_residencia,co_regional_residencia,co_distrito_residencia,co_bairro_residencia,no_bairro_residencia,co_logradouro_residencia,nome_logradouro_residencia,nu_cep_residencia,tp_zona_residencia,co_pais_residencia,dt_investigacao,co_cbo_ocupacao,febre,mialgia,cefaleia,exantema,vomito,nausea,dor_costas,conjutivite,artrite,artralgia,petequia_n,leucopenia,laco,dor_retro,diabetes,hematolog,hepatopat,renal,hipertensao,acido_pept,auto_imune,dt_chil_s1,dt_chil_s2,dt_prnt,res_chiks1,res_chiks2,resul_prnt,dt_coleta_exame,tp_result_exame,dt_coleta_NS1,Tp_result_NS1,dt_coleta_isolamento,tp_result_isolamento,dt_coleta_rtpcr,tp_result_rtpcr,tp_sorotipo,tp_result_histopatologia,tp_result_imunohistoquimica,st_ocorreu_hospitalizacao,dt_internacao,co_uf_hospital,co_municipio_hospital,tp_autoctone_residencia,co_uf_infeccao,co_pais_infeccao,co_municipio_infeccao,co_distrito_infeccao,co_bairro_infeccao,no_bairro_infeccao,tp_classificacao_final,tp_criterio_confirmacao,st_doenca_trabalho,clinc_chik,tp_evolucao_caso,dt_obito,dt_encerramento,alrm_hipot,alrm_plaq,alrm_vom,alrm_sang,alrm_hemat,alrm_abdom,alrm_letar,alrm_hepat,alrm_liq,dt_alrm,grav_pulso,grav_conv,grav_ench,grav_insuf,grav_taqui,grav_extre,grav_hipot,grav_hemat,grav_melen,grav_metro,grav_sang,grav_ast,grav_mioc,grav_consc,grav_orgao,dt_grav,mani_hemor,epistaxe,gengivo,metro,petequias,hematura,sangram,laco_n,plasmatico,evidencia,plaq_menor,con_fhd,complica
0,3545397,2,A90,2020-01-01,202001,2020,26,260790,1497.0,6618464,2019-12-30,202001,1992-02-10,4027,M,6.0,4,9.0,26,261160,1497,121.0,81.0,SAN MARTIN,NaN,RUA APULCRO ASSUNCAO,50761010.0,9.0,1,2020-01-01,NaN,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,4.0,NaN,4.0,NaN,4.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,NaN,NaN,1.0,NaN,2020-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3507054,2,A90,2020-01-01,202001,2020,26,261160,1497.0,671,2019-12-30,202001,1984-10-04,4035,F,9.0,4,9.0,26,261160,1497,547.0,88.0,COHAB,NaN,RUA FLAVIANO BARBOSA DE OLIVEIRA,NaN,1.0,1,2020-01-01,NaN,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,4.0,NaN,4.0,NaN,4.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,2.0,NaN,NaN,1.0,NaN,2020-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3520539,2,A90,2020-01-02,202001,2020,26,261160,1497.0,7958838,2019-12-31,202001,1995-03-11,4024,F,5.0,9,9.0,26,261160,1497,119.0,59.0,DOIS IRMAOS,NaN,RUA MANOEL DE MEDEIROS,NaN,1.0,1,2020-01-02,NaN,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,2.0,NaN,NaN,1.0,NaN,2020-02-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3521380,2,A90,2020-01-03,202001,2020,26,261160,1497.0,6530389,2019-12-29,202001,1995-02-27,4024,M,6.0,9,9.0,26,261160,1497,120.0,70.0,TORROES,NaN,RUA DANDARA,50650050.0,1.0,1,2020-01-03,NaN,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,26.0,1.0,261160.0,120.0,70.0,TORROES,10.0,2.0,NaN,NaN,1.0,NaN,2020-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

### Observações iniciais — Dengue 2020

Nesta etapa, buscamos identificar:

- Quantidade de registros
- Principais variáveis disponíveis
- Presença de valores ausentes
- Campos territoriais (bairro, distrito, município)
- Campos temporais (ano, mês, data de notificação)

Essas observações irão orientar as próximas etapas de limpeza e padronização.


In [17]:
bairros = read_csv_br("../data/raw/tabelas_auxiliares/tabela-de-bairros.csv")
distritos = read_csv_br("../data/raw/tabelas_auxiliares/tabela-distrito.csv")
agravos = read_csv_br("../data/raw/tabelas_auxiliares/tabela-dos-agravos.csv")
ufs = read_csv_br("../data/raw/tabelas_auxiliares/tabela-uf.csv")

bairros.head(10)

,Nº Localidade,Nome Localidade,Nome Município
0,132,AFLITOS,RECIFE
1,779,AFOGADOS,RECIFE
2,337,AGUA FRIA,RECIFE
3,493,ALTO DO MANDU,RECIFE
4,540,ALTO JOSE BONIFACIO,RECIFE
5,523,ALTO JOSE DO PINHO,RECIFE
6,949,ALTO SANTA TEREZINHA,RECIFE
7,507,APIPUCOS,RECIFE
8,850,AREIAS,RECIFE
9,329,ARRUDA,RECIFE


In [14]:
distritos.head(10)

,Código Distrito,Distrito,Código Município,Município
0,117,0001- CENTRO EXPANDIDO,261160,RECIFE
1,118,0002- ENCRUZILHADA-BEBERIBE,261160,RECIFE
2,119,0003- CASA AMARELA- DOIS IRMAOS,261160,RECIFE
3,120,0004- CAXANGA-VARZEA,261160,RECIFE
4,121,0005- AFOGADOS-TEJIPIO,261160,RECIFE
5,122,0006- IBURA-BOA VIAGEM,261160,RECIFE


In [18]:
agravos.head(10)

,Código CID,Agravo
0,A92.8,FEBRE PELO VIRUS ZIKA
1,A92.0,FEBRE DE CHIKUNGUNYA
2,A90,DENGUE
3,A23,BRUCELOSE
4,A69.2,DOENCA DE LYME
5,A92.8,FEBRE PELO VIRUS ZIKA
6,B74.9,FILARIOSE NAO ESPECIFICADA


In [20]:
ufs.head(28)

,codigo,sigla,descricao
0,12,AC,ACRE
1,27,AL,ALAGOAS
2,16,AP,AMAPA
3,13,AM,AMAZONAS
4,29,BA,BAHIA
5,23,CE,CEARA
6,53,DF,DISTRITO FEDERAL
7,32,ES,ESPIRITO SANTO
8,52,GO,GOIAS
9,21,MA,MARANHAO


## Dados demográficos — Censo 2022 (IBGE)

Os dados do Censo 2022 serão utilizados futuramente para:

- Normalizar os casos por população
- Analisar desigualdades territoriais
- Relacionar condições domiciliares e crescimento populacional com a incidência das doenças


In [24]:
demografia_territorio = read_csv_br("../data/raw/demografia/Agregados_por_bairros_basico_BR.csv")
demografia_territorio.head(10)


,CD_BAIRRO,NM_BAIRRO,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_SUBDIST,NM_SUBDIST,CD_NU,NM_NU,CD_AGLOM,NM_AGLOM,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,v0001,v0002,v0003,v0004,v0005,v0006,v0007
0,1100015001,Centro,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,11000150500,NaN,.,NaN,.,NaN,1102,Ji-Paraná,110005,Cacoal,.,NaN,"0,6282427",1213,637,633,4,"2,5","0,0146",480
1,1100015002,Liberdade,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,11000150500,NaN,.,NaN,.,NaN,1102,Ji-Paraná,110005,Cacoal,.,NaN,"0,9519164",1029,488,488,0,"2,6","0,0281",391
2,1100015003,Cidade Alta,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,11000150500,NaN,.,NaN,.,NaN,1102,Ji-Paraná,110005,Cacoal,.,NaN,"0,5963224",1288,602,600,2,"2,6","0,0040",499
3,1100015005,Princesa Isabel,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,11000150500,NaN,.,NaN,.,NaN,1102,Ji-Paraná,110005,Cacoal,.,NaN,"1,9519548",2846,1242,1242,0,"2,7","0,0179",1060
4,1100015006,Redondo,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,11000150500,NaN,.,NaN,.,NaN,1102,Ji-Paraná,110005,Cacoal,.,NaN,"1,4534337",2365,990,990,0,"2,7","0,0619",873
5,1100015007,Tucanos,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,11000150500,NaN,.,NaN,.,NaN,1102,Ji-Paraná,110005,Cacoal,.,NaN,"0,2255128",325,138,138,0,"2,8","0,0000",116
6,1100015008,Santa Felicidade,1,Norte,11,Rondônia,1100015,Alta Floresta D'Oeste,110001505,Alta Floresta D'Oeste,11000150500,NaN,.,NaN,.,NaN,1102,Ji-Paraná,110005,Cacoal,.,NaN,"0,7610610",2142,1006,1004,2,"2,7","0,0077",777
7,1100023001,Setor Apoio Rodoviário,1,Norte,11,Rondônia,1100023,Ariquemes,110002305,Ariquemes,11000230500,NaN,.,NaN,.,NaN,1101,Porto Velho,110002,Ariquemes,.,NaN,"0,3856944",477,197,194,3,"2,8","0,1637",171
8,1100023002,Apoio Social,1,Norte,11,Rondônia,1100023,Ariquemes,110002305,Ariquemes,11000230500,NaN,.,NaN,.,NaN,1101,Porto Velho,110002,Ariquemes,.,NaN,"0,4284931",959,417,417,0,"2,8","0,0235",341
9,1100023003,Área Especial 02,1,Norte,11,Rondônia,1100023,Ariquemes,110002305,Ariquemes,11000230500,NaN,.,NaN,.,NaN,1101,Porto Velho,110002,Ariquemes,.,NaN,"0,1215491",659,294,294,0,"2,5","0,0303",264


In [29]:
Recife_demografia_territorio = demografia_territorio[demografia_territorio["NM_MUN"] == "Recife"
]

Recife_demografia_territorio = Recife_demografia_territorio.reset_index(drop=True)

Recife_demografia_territorio.head(10)

,CD_BAIRRO,NM_BAIRRO,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_SUBDIST,NM_SUBDIST,CD_NU,NM_NU,CD_AGLOM,NM_AGLOM,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,v0001,v0002,v0003,v0004,v0005,v0006,v0007
0,2611606001,Recife,2,Nordeste,26,Pernambuco,2611606,Recife,261160605,Recife,26116060518,RPA 01,.,NaN,.,NaN,2601,Recife,260001,Recife,2611606,Recife/PE,"1,2355920",541,224,222,2,"2,7","0,0941",202
1,2611606002,Santo Amaro,2,Nordeste,26,Pernambuco,2611606,Recife,261160605,Recife,26116060518,RPA 01,.,NaN,.,NaN,2601,Recife,260001,Recife,2611606,Recife/PE,"3,3778321",27103,11661,11629,32,"2,7","0,0373",10054
2,2611606003,Boa Vista,2,Nordeste,26,Pernambuco,2611606,Recife,261160605,Recife,26116060518,RPA 01,.,NaN,.,NaN,2601,Recife,260001,Recife,2611606,Recife/PE,"1,7697463",13147,8328,8266,62,"2,1","0,0614",6137
3,2611606004,Soledade,2,Nordeste,26,Pernambuco,2611606,Recife,261160605,Recife,26116060518,RPA 01,.,NaN,.,NaN,2601,Recife,260001,Recife,2611606,Recife/PE,"0,3234122",2081,1553,1545,8,"1,9","0,0443",1084
4,2611606005,Santo Antônio,2,Nordeste,26,Pernambuco,2611606,Recife,261160605,Recife,26116060518,RPA 01,.,NaN,.,NaN,2601,Recife,260001,Recife,2611606,Recife/PE,"0,5177116",447,324,324,0,"2,0","0,0175",228
5,2611606006,Paissandu,2,Nordeste,26,Pernambuco,2611606,Recife,261160605,Recife,26116060518,RPA 01,.,NaN,.,NaN,2601,Recife,260001,Recife,2611606,Recife/PE,"0,3450973",568,348,348,0,"2,1","0,0582",275
6,2611606007,Ilha do Leite,2,Nordeste,26,Pernambuco,2611606,Recife,261160605,Recife,26116060518,RPA 01,.,NaN,.,NaN,2601,Recife,260001,Recife,2611606,Recife/PE,"0,2622270",990,757,755,2,"2,0","0,1963",489
7,2611606008,Coelhos,2,Nordeste,26,Pernambuco,2611606,Recife,261160605,Recife,26116060518,RPA 01,.,NaN,.,NaN,2601,Recife,260001,Recife,2611606,Recife/PE,"0,4251229",5923,2538,2525,13,"2,7","0,0531",2204
8,2611606009,Ilha Joana Bezerra,2,Nordeste,26,Pernambuco,2611606,Recife,261160605,Recife,26116060522,RPA 05,.,NaN,.,NaN,2601,Recife,260001,Recife,2611606,Recife/PE,"1,1688231",12257,4766,4765,1,"2,9","0,0371",4264
9,2611606010,Cabanga,2,Nordeste,26,Pernambuco,2611606,Recife,261160605,Recife,26116060518,RPA 01,.,NaN,.,NaN,2601,Recife,260001,Recife,2611606,Recife/PE,"0,5651980",1831,835,834,1,"2,7","0,0841",666


In [28]:
metadados_recife = read_csv_br("../data/raw/metadados/metadados_recife_demografia_territorio.csv")
metadados_recife.head(20)

,ï»¿Tema,VariÃ¡vel,DescriÃ§Ã£o
0,BÃ¡sico,V0001,Total de pessoas
1,BÃ¡sico,V0002,Total de DomicÃ­lios (DPPO + DPPV + DPPUO + DP...
2,BÃ¡sico,V0003,Total de DomicÃ­lios Particulares (DPPO + DPPV...
3,BÃ¡sico,V0004,Total de DomicÃ­lios Coletivos (DCCM + DCSM)
4,BÃ¡sico,V0005,MÃ©dia de moradores em DomicÃ­lios Particulare...
5,BÃ¡sico,V0006,Percentual de DomicÃ­lios Particulares Ocupado...
6,BÃ¡sico,V0007,Total de DomicÃ­lios Particulares Ocupados (DP...


## Conclusão deste notebook

Este notebook teve como foco a **compreensão inicial das bases de dados** utilizadas no projeto.

### Próximos passos:
- Padronização de colunas entre anos e agravos
- Consolidação dos arquivos anuais
- Integração com dados demográficos
- Análises espaciais e temporais mais aprofundadas

Os próximos notebooks tratarão dessas etapas.
